In [1]:
fns = ["labelmaps/sub-122017_L_bg-0_.nii.gz",
    "labelmaps/sub-122017_R_bg-0_flip.nii.gz",
    "labelmaps/sub-152017_L_bg-0_.nii.gz",
    "labelmaps/sub-152017_R_bg-0_flip.nii.gz",
    "labelmaps/sub-bbhist_L_bg-0_.nii.gz",
    "labelmaps/sub-bbhist_R_bg-0_flip.nii.gz",
    "labelmaps/sub-pli3d_L_bg-0_.nii.gz",]

In [2]:
import os
import shutil
import subprocess
from glob import glob

# Create clean output directory
output_dir = 'layertemplate'
os.makedirs(output_dir, exist_ok=True)

# Clear existing contents
for f in glob(os.path.join(output_dir, '*')):
    os.remove(f)

# Open templatelist file for writing
csv_path = os.path.join(output_dir, 'templatelist.csv')
with open(csv_path, 'w') as csv_file:
    for i, fn in enumerate(fns):
        sub, hemi = os.path.basename(fn).split('_')[:2]

        out_fn = os.path.join(output_dir, f'tmp_{i}.nii.gz')

        if hemi[-1] == 'L':
            subprocess.run(['c3d', fn, '-flip', 'x', '-o', out_fn], check=True)
        else:
            shutil.copy(fn, out_fn)

        # Smooth each layer in the volume
        split_cmd = [
            'c3d', out_fn,
            '-split', '-foreach', '-smooth', '0.5x0.5x0.5mm', '-endfor',
            '-oo', os.path.join(output_dir, f'sub-{i}_label_%02d.nii.gz')
        ]
        subprocess.run(split_cmd, check=True)

        # Find smoothed label outputs
        label_paths = sorted(glob(os.path.join(output_dir, f'sub-{i}_label_*.nii.gz')))
        for lbl in label_paths:
            csv_file.write(f"{lbl},")
        csv_file.write("\n")
        for f in glob(os.path.join(output_dir, 'tmp_*.nii.gz')):
            os.remove(f)


Exception caught of type 16ConvertException
  Exception detail: Unable to read image labelmaps/sub-122017_L_bg-0_.nii.gz; IO factory can not create IO object.


CalledProcessError: Command '['c3d', 'labelmaps/sub-122017_L_bg-0_.nii.gz', '-flip', 'x', '-o', 'layertemplate/tmp_0.nii.gz']' returned non-zero exit status 255.

In [ ]:
!export ANTSPATH=/opt/minc/1.9.18/bin/
cmd = f"antsMultivariateTemplateConstruction2.sh -d 3 -o layertemplate/template_ layertemplate/templatelist.csv"
!{cmd}